In [1]:
from myfunctions import *
from sklearn.feature_extraction.text import CountVectorizer
from deep_pytorch import *
from sklearn.preprocessing import StandardScaler
import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

/home/nmduy/anaconda3/envs/graph/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.ranking module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
data_full = pd.read_json('fake_news.json', lines=True)
data_full = data_full.drop(columns=['article_link']) # remove link column
df_train_f, df_test = split_dataframe(data_full, test_size=0.25, seed=1509)
df_train, df_validate = split_dataframe(df_train_f, test_size=0.2, seed=1309)

# Proportion of each subsets
list_label = df_train['is_sarcastic'].tolist()
numb_total = len(list_label)
numb_sarcastic = np.sum(np.asarray(list_label))
numb_not_sarcastic = numb_total - numb_sarcastic
print(f'===== TRAINING SAMPLES =====\nTotal Sample: {numb_total}\nSarcastic: {numb_sarcastic} ({np.round(numb_sarcastic/numb_total*100,2)}%)\nNot Sarcastic: {numb_not_sarcastic} ({np.round(numb_not_sarcastic/numb_total*100,2)}%)')

list_label = df_validate['is_sarcastic'].tolist()
numb_total = len(list_label)
numb_sarcastic = np.sum(np.asarray(list_label))
numb_not_sarcastic = numb_total - numb_sarcastic
print(f'===== VALIDATING SAMPLES =====\nTotal Sample: {numb_total}\nSarcastic: {numb_sarcastic} ({np.round(numb_sarcastic/numb_total*100,2)}%)\nNot Sarcastic: {numb_not_sarcastic} ({np.round(numb_not_sarcastic/numb_total*100,2)}%)')

list_label = df_test['is_sarcastic'].tolist()
numb_total = len(list_label)
numb_sarcastic = np.sum(np.asarray(list_label))
numb_not_sarcastic = numb_total - numb_sarcastic
print(f'===== TESTING SAMPLES =====\nTotal Sample: {numb_total}\nSarcastic: {numb_sarcastic} ({np.round(numb_sarcastic/numb_total*100,2)}%)\nNot Sarcastic: {numb_not_sarcastic} ({np.round(numb_not_sarcastic/numb_total*100,2)}%)')

===== TRAINING SAMPLES =====
Total Sample: 18316
Sarcastic: 8726 (47.64%)
Not Sarcastic: 9590 (52.36%)
===== VALIDATING SAMPLES =====
Total Sample: 4579
Sarcastic: 2181 (47.63%)
Not Sarcastic: 2398 (52.37%)
===== TESTING SAMPLES =====
Total Sample: 5724
Sarcastic: 2727 (47.64%)
Not Sarcastic: 2997 (52.36%)


In [3]:
data_train = df_train
data_train['headline_s1'] = data_train.headline.apply(lambda row: remove_symbol(row))
data_train['headline_s2'] = data_train.headline_s1.apply(lambda row: lemmatize_word(row, 'v'))
data_train['headline_s2'] = data_train.headline_s2.apply(lambda row: lemmatize_word(row, 'n'))
data_train = data_train.drop(columns=['headline', 'headline_s1'])

data_val = df_validate
data_val['headline_s1'] = data_val.headline.apply(lambda row: remove_symbol(row))
data_val['headline_s2'] = data_val.headline_s1.apply(lambda row: lemmatize_word(row, 'v'))
data_val['headline_s2'] = data_val.headline_s2.apply(lambda row: lemmatize_word(row, 'n'))
data_val = data_val.drop(columns=['headline', 'headline_s1'])

data_train_rmsw = data_train.copy()
data_train_rmsw['headline_s3'] = data_train_rmsw.headline_s2.apply(lambda row: remove_stop_words(row))
data_train_rmsw = data_train_rmsw.drop(columns=['headline_s2'])

data_val_rmsw = data_val.copy()
data_val_rmsw['headline_s3'] = data_val_rmsw.headline_s2.apply(lambda row: remove_stop_words(row))
data_val_rmsw = data_val_rmsw.drop(columns=['headline_s2'])

# BOW Entire Vocab

In [4]:
vectorizer = CountVectorizer()
all_string = data_train.headline_s2.tolist()
all_string_in_one = ' '.join(all_string)
# tokenize and build vocab
vectorizer.fit([all_string_in_one])
# encode document
vector = vectorizer.transform(data_train.headline_s2.tolist())
# summarize encoded vector
vector = vector.toarray()
data_train = data_train.drop(columns=['headline_s2'])

for idx in range(vector.shape[1]):
    if idx % 1000 == 0:
        print(f'Processing FT_{idx}')
    data_train[f"ft_{idx}"] = vector[:,idx]
    
# encode document
vector = vectorizer.transform(data_val.headline_s2.tolist())
# summarize encoded vector
vector = vector.toarray()
data_val = data_val.drop(columns=['headline_s2'])

for idx in range(vector.shape[1]):
    if idx % 1000 == 0:
        print(f'Processing FT_{idx}')
    data_val[f"ft_{idx}"] = vector[:,idx]

Processing FT_0
Processing FT_1000
Processing FT_2000
Processing FT_3000
Processing FT_4000
Processing FT_5000
Processing FT_6000
Processing FT_7000
Processing FT_8000
Processing FT_9000
Processing FT_10000
Processing FT_11000
Processing FT_12000
Processing FT_13000
Processing FT_14000
Processing FT_15000
Processing FT_16000
Processing FT_17000
Processing FT_18000
Processing FT_0
Processing FT_1000
Processing FT_2000
Processing FT_3000
Processing FT_4000
Processing FT_5000
Processing FT_6000
Processing FT_7000
Processing FT_8000
Processing FT_9000
Processing FT_10000
Processing FT_11000
Processing FT_12000
Processing FT_13000
Processing FT_14000
Processing FT_15000
Processing FT_16000
Processing FT_17000
Processing FT_18000


In [5]:
X_train = data_train.drop(columns=['is_sarcastic'])
y_train = data_train['is_sarcastic']
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

X_validate = data_val.drop(columns=['is_sarcastic'])
y_validate = data_val['is_sarcastic']
X_validate = np.asarray(X_validate)
y_validate = np.asarray(y_validate)

feature_names = list(data_val.drop(columns=['is_sarcastic']).columns)

## NB

In [12]:
model_nb = GaussianNB()
model_nb.fit(X_train, y_train)

validate_pred = model_nb.predict_proba(X_validate)
validate_pred = validate_pred[:,1]
metrics = calculate_metric(y_validate, validate_pred)
metrics

{'accuracy': 0.6641188032321468,
 'precision': 0.7620211898940505,
 'recall': 0.4287024300779459,
 'f1': 0.5487089201877935,
 'tp': 935,
 'tn': 2106,
 'fp': 292,
 'fn': 1246,
 'auc': 0.6559682740354851}

## RF

In [13]:
hypara = {
    'random_state': 1509,
    'max_depth': 20,
    'n_estimators': 300,
    'min_samples_split': 7,
    'min_samples_leaf': 2,
}
model_rf = RandomForestClassifier(**hypara)
model_rf.fit(X_train, y_train);

In [14]:
validate_pred = model_rf.predict_proba(X_validate)
validate_pred = validate_pred[:,1]
metrics = calculate_metric(y_validate, validate_pred)
metrics

{'accuracy': 0.7532212273422144,
 'precision': 0.8631651693158259,
 'recall': 0.572673085740486,
 'f1': 0.6885336273428886,
 'tp': 1249,
 'tn': 2200,
 'fp': 198,
 'fn': 932,
 'auc': 0.8724176573860457}

## SVM (Too Long)

In [ ]:
hypara = {
    'C': 100,
    'random_state': 1509,
    'max_iter':10000,
    'tol' : 1e-5
}
model_svm = SVC(**hypara)
model_svm.fit(X_train, y_train);

validate_pred = model_svm.predict(X_validate)
metrics = calculate_metric(y_validate, validate_pred)
metrics

## KNN

In [6]:
hypara = {
    'n_neighbors': 200,
    'weights': 'distance'
}
model_knn = KNeighborsClassifier(**hypara)
model_knn.fit(X_train, y_train);

validate_pred = model_knn.predict_proba(X_validate)
validate_pred = validate_pred[:,1]
metrics = calculate_metric(y_validate, validate_pred)
metrics

{'accuracy': 0.5383271456649924,
 'precision': 0.958904109589041,
 'recall': 0.03209536909674461,
 'f1': 0.062111801242236024,
 'tp': 70,
 'tn': 2395,
 'fp': 3,
 'fn': 2111,
 'auc': 0.7645183457558051}

## Logistic Regression

In [ ]:
hypara = {
    'penalty': 'l1',
    'C': 5,
    'max_iter': 5000,
    'tol': 1e-5,
    'solver': 'saga'
}

model_lr = LogisticRegression(**hypara)
model_lr.fit(X_train, y_train);
validate_pred = model_lr.predict_proba(X_validate)
validate_pred = validate_pred[:,1]
metrics = calculate_metric(y_validate, validate_pred)
metrics